In [20]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from pathlib import Path
import os


In [21]:
def busqueda_link_artista(artista: str) -> str:
    """
    Función encargada de traer las canciones del artista seleccionado
    """
    artista_link = artista.replace(" ", "-").lower()
    link = f"https://www.letras.com/{artista_link}/mais_acessadas.html"
    response = requests.get(link)
    return BeautifulSoup(response.content, "html.parser")

In [22]:
def busqueda_lista_canciones(artista: str) -> list[str]:
    """
    Función encargada de buscar los path donde se encuentran las canciones de cada artista
    """
    first_soup = busqueda_link_artista(artista)
    canciones = first_soup.find_all(class_="songList-table-row --song isVisible")  # Encuentra el path con otra información #OJO AQUI, PUEDEN CAMBIAR EL COSO / song-name
    patron = r'"\/(.*?)\/"'
    path_canciones = [re.findall(patron, str(cancion))[0] for cancion in canciones]  # Se limpia el path de busqueda
    return path_canciones

In [23]:
def extraccion_data(soup: str) -> list[str]:
    """
    Función encargada de sacar Género, Artista, Título y Canción
    """

    # Sección encargada de extraer genero, artista y titulo

    gen_art_tit = soup.find_all(itemprop="name")  # aqui se encuentra casi todo lo de genero, artista y titulo
    patron = r">(.*?)<"  # solo extrae lo que está dentro de eso
    vector = [re.findall(patron, str(texto))[0] for texto in gen_art_tit[1:]]  # Genera vector de lo mencionado anteriormente, findall trae un vector por eso solo se trae la pos 0

    # print(soup.find("body"))

    # Sección encargada de extraer la canción

    cancion = soup.find_all(class_="lyric-original")[0]  # OJO QUE LO CAMBIAN - cnt-letra
    patron_etiqueta = r"<[^>]+>"
    patron_saltos = r"\n \n"
    texto_sin_etiquetas = re.sub(patron_etiqueta, "\n", str(cancion))
    texto_final = re.sub(patron_saltos, "", texto_sin_etiquetas)
    vector.append(texto_final)

    # # Crear archivo TXT
    # nombre_archivo = carpeta_artista + "/" + str(vector[2]) + ".txt"
    # with open(nombre_archivo, "w") as archivo:
    #     archivo.write(vector[3])

    return vector


In [24]:
def recoleccion_informacion_cancion(path: list, cantidad_canciones_extraer: int) -> list[str]:
    """
    A partir de los path generados procede a guardar en un arreglo toda la informacion inicial necesaria para el proyecto
    """
    vector_canciones = []
    urls = ["https://www.letras.com/" + path_unico + "/" for path_unico in path]
    for url in urls[:cantidad_canciones_extraer]:
        response = requests.get(url)
        html_content = response.content
        soup = BeautifulSoup(html_content, "html.parser")
        vector_canciones.append(extraccion_data(soup))
    return vector_canciones


In [25]:
artista = "aitana"
canciones_scrappear_por_artista = 50

In [26]:
path = busqueda_lista_canciones(artista)
canciones_artistas = recoleccion_informacion_cancion(path, canciones_scrappear_por_artista)

In [27]:
canciones_artistas

[['Pop',
  'Aitana',
  '+ (MÁS) (part. Cali y El Dandee)',
  'Que no volveré, que no volverás\nQue después de un Sol no te veré más\nDime que es mentira, que me lo soñé\nQue tú ya no te vas\n\nQue, a partir de hoy, todo es recordar\nNo te olvidaré, no me olvidarás\nDime que no es cierto\nY que este amor tan grande no se acabará\n\nHoy no me voy a dormir\nPara que al reloj no le pasen las horas\nSonrisas por fuera aunque por dentro lloras\nYo voy a quedarme y tú te vas a ir (tú te vas a ir)\n\n¿Qué más te puedo decir?\nSi el primer amor durara para siempre\nSobrarán recuerdos, faltará tenerte\nDejas una historia en mí por escribir\n\nSi yo te quiero, te quiero y te quiero\nY por tu culpa, febrero me duele de más\n¿Dónde guardaré este amor si tú te vas?\nSi no es mentira que te echo de menos\nSi yo no te echo de menos, te extraño de más\nY aunque sé muy bien que tú no volverás\n¿Cómo pretendo no echarte de menos si te amé de más?\n\nMás te amo, menos tiempo, más te alejas, más lo siento\

In [28]:
if not os.path.exists(f"./canciones/{artista}"):
    os.makedirs(f"./canciones/{artista}")

In [29]:
for info in canciones_artistas:
    titulo = info[2].replace("¿", "").replace("?", "")
    cancion = info[3]

    file = open(f"./canciones/{artista}/{artista.lower()}-{titulo.lower()}.txt", "w+", encoding="utf-8")
    file.write(cancion)
    file.close()